### Secondary Data Cleaning

This file is to clean and process secondary data (GDP and MFN tarrif data)
which involves following steps

1. Standardizing country names
2. Clean the data
3. Converting wide format to long format
4. Merging MFN Tarrif and GDP data
5. Writing final data to a csv to be merged with primary data

In [39]:
# import all the modules needed
import re
import pandas as pd
import numpy as np
import country_converter as coco

#### Define helper functions to clean and process data

In [40]:
def standardized_country(countries):
    """
    Converts a list of country names or codes to their standardized short names.

    Parameters
    ----------
    countries : list or str
        A list of country names or codes, or a single country name/code, to be standardized.

    Returns
    -------
    list or str
        The standardized short names of the input countries. 
        If a single country is provided, returns a string.
        If a list is provided, returns a list of strings.

    Notes
    -----
    This function uses the `coco` library to perform the conversion. 
    The output format is the short name of each country or `not found` 
    if country name/code is not recognized.
    """
    return coco.convert(names=countries, to='name_short')

In [41]:
def clean_secondary_data(df, country_col, value_col, cols_to_keep,
                         start_year=2015, end_year=2022, not_found_label="not found"):
    """
    Clean and preprocess a dataframe by standardizing country names, 
    filtering columns and rows, and melting into long format.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing data with country and year columns.
    country_col : str
        The name of the column in `df` containing country names to be standardized.
    value_col : str
        The name of the value column in the output DataFrame after melting.
    cols_to_keep : list of str
        The list of columns to keep before melting (e.g., standardized country and year columns).
    start_year : int, optional
        The first year to consider when dropping rows with missing data.
    end_year : int, optional
        The last year to consider when dropping rows with missing data.
    not_found_label : str, optional (default="not found")
        The label used to identify unrecognized countries.

    Returns
    -------
    pd.DataFrame
        A cleaned DataFrame with standardized country names, 
        a 'Year' column, and a column for data values.

    Notes
    -----
    Steps performed:
    1. Drop rows with no data between `start_year` and `end_year`.
    2. Standardize country names using `standardized_country`.
    3. Drop rows where the standardized name equals `not_found_label`.
    4. Retain only the specified columns (`cols_to_keep`).
    5. Melt the DataFrame into long format with `Year`, `standardized_country`, and values.
    """
    # Drop rows with no data in the given year range
    year_cols = [str(y) for y in range(start_year, end_year + 1)]
    df = df.dropna(subset=year_cols, how="all")

    # Standardize country names
    df = df.assign(standardized_country=standardized_country(df[country_col]))

    # Drop rows where country standardization failed
    df = df.loc[df["standardized_country"] != not_found_label, :]

    # Keep only relevant columns
    df = df.loc[:, cols_to_keep]

    # Melt into long format
    melted_df = pd.melt(df, id_vars=["standardized_country"], var_name="Year", value_name=value_col)

    return melted_df

In [42]:
def get_cols_to_keep(start_year=2008, end_year=2022, string_col="standardized_country"):
    """
    Generate a list of columns to keep for data cleaning.

    Parameters
    ----------
    start_year : int, optional
        The first year to include in the list of columns.
    end_year : int, optional
        The last year to include in the list of columns.
    string_cols : list of str, optional
        A list of string column names to include before the year columns.

    Returns
    -------
    list of str
        A list of column names including 'standardized_country' and years
        from `start_year` to `end_year`.
    """
    years = [str(year) for year in range(start_year, end_year + 1)]
    return [string_col] + years

#### Process GDP Data

We have two GDP datasets captured in separate files:

1. GDP in Current USD:
    This indicator is expressed in current prices, meaning no adjustment has been made
    to account for price changes over time. This indicator is expressed in United States dollars.
2. GDP in 2015-Adjusted USD:
    This indicator is expressed in constant prices, meaning the series has been adjusted
    to account for price changes over time. The reference year for this adjustment is 2015.
    This indicator is expressed in United States dollars.
    It removes the effects of price changes and inflation.

Load both GDP datasets for initial EDA; decide later which measure to use based on analysis goals.


In [43]:
# Read GDP data files and skip the last 5 footer rows
raw_gdp_df = pd.read_csv("./../../data/raw/secondary/GDP.csv",
                         skipfooter=5,
                         engine="python")
raw_gdp_2015_adj_df = pd.read_csv("./../../data/raw/secondary/GDP_2015_adjusted.csv",
                                  skipfooter=5,
                                  engine="python")

print(raw_gdp_df.head(2))
print(raw_gdp_2015_adj_df.head(2))

         Series Name     Series Code Country Name Country Code 1960 [YR1960]  \
0  GDP (current US$)  NY.GDP.MKTP.CD  Afghanistan          AFG            ..   
1  GDP (current US$)  NY.GDP.MKTP.CD      Albania          ALB            ..   

  1961 [YR1961] 1962 [YR1962] 1963 [YR1963] 1964 [YR1964] 1965 [YR1965]  ...  \
0            ..            ..            ..            ..            ..  ...   
1            ..            ..            ..            ..            ..  ...   

      2015 [YR2015]     2016 [YR2016]     2017 [YR2017]     2018 [YR2018]  \
0  19134221644.7325  18116572395.0772  18753456497.8159  18053222687.4126   
1  11470171826.9575  11988668784.6628  13258268435.6048  15379508329.7568   

      2019 [YR2019]     2020 [YR2020]     2021 [YR2021]     2022 [YR2022]  \
0  18799444490.1128  19955929052.1496  14259995441.0759  14497243872.1337   
1  15585111614.0376  15241455556.3737  18032009220.4405  19017244116.7203   

      2023 [YR2023]     2024 [YR2024]  
0  17152234636

In [44]:
# rows after 216 are not countries but groups such as region, income groups.
# Only country level data is required for our analysis. Hence select only countries
raw_gdp_df = raw_gdp_df.loc[0:216, :]
raw_gdp_2015_adj_df = raw_gdp_2015_adj_df.loc[0:216, :]

In [45]:
# rename columns so that it looks consistent in all sources of data
# remove the text in square brackets from the column names
# e.g. "2008 [YR2008]" to "2008"
raw_gdp_df = raw_gdp_df.rename(
    columns=lambda col: re.sub(r" \[YR\d{4}\]$", '', col)
)

raw_gdp_2015_adj_df = raw_gdp_2015_adj_df.rename(
    columns=lambda col: re.sub(r" \[YR\d{4}\]$", '', col)
)

In [46]:
# get the columns to keep using the function
required_columns = get_cols_to_keep(2008,2022)

# clean the gdp dataframe by passing the required parameters to the function
cleaned_gdp_df = clean_secondary_data(
    df=raw_gdp_df,
    country_col="Country Name",
    value_col="GDP",
    cols_to_keep=required_columns)

# clean the gdp 2015 adjusted dataframe by passing the required parameters to the function
cleaned_gdp_2015_adj_df = clean_secondary_data(
    df=raw_gdp_2015_adj_df,
    country_col="Country Name",
    value_col="GDP_2015_adj",
    cols_to_keep=required_columns)

# print the dataframes after cleaning
print("\n\n")
print(cleaned_gdp_df.head(2))
print("\n\n")
print(cleaned_gdp_2015_adj_df.head(2))

Channel Islands not found in regex
Channel Islands not found in regex





  standardized_country  Year               GDP
0          Afghanistan  2008  10109297047.5432
1              Albania  2008  13258506375.4518



  standardized_country  Year      GDP_2015_adj
0          Afghanistan  2008  11060395115.6809
1              Albania  2008  9861658465.33864


In [47]:
# merge both GDP dataframes on standardized country and year
final_gdp_df = pd.merge(cleaned_gdp_df, cleaned_gdp_2015_adj_df,
                        on=["standardized_country", "Year"],
                        how="inner")
final_gdp_df.head(2)

,standardized_country,Year,GDP,GDP_2015_adj
0,Afghanistan,2008,10109297047.5432,11060395115.6809
1,Albania,2008,13258506375.4518,9861658465.33864


In [48]:
# write the final gdp dataframe to a csv file
final_gdp_df.to_csv("./../../data/processed/final_gdp_df.csv", index=False)

#### Process MFN tarrif data

https://wits.worldbank.org/CountryProfile/en/Country/USA/StartYear/1991/EndYear/2022/TradeFlow/Import/Partner/BY-COUNTRY/Indicator/MFN-WGHTD-AVRG

https://wits.worldbank.org/CountryProfile/en/Country/USA/StartYear/1991/EndYear/2022/TradeFlow/Import/Partner/BY-COUNTRY/Indicator/MFN-SMPL-AVRG

https://wits.worldbank.org/CountryProfile/en/Country/BY-COUNTRY/StartYear/1988/EndYear/2022/TradeFlow/Import/Partner/USA/Indicator/MFN-SMPL-AVRG

https://wits.worldbank.org/CountryProfile/en/Country/BY-COUNTRY/StartYear/1988/EndYear/2022/TradeFlow/Import/Partner/USA/Indicator/MFN-WGHTD-AVRG



We are using four datasets from the World Bank WITS portal on Most Favored Nation (MFN) tariffs.
- **By US (tariffs imposed by the US on imports):**
  - Weighted Average (United States MFN Weighted Average by country.xlsx)
  - Simple Average (United States MFN Simple Average by country.xlsx)

- **On US (tariffs faced by US exports in partner countries):**
  - Weighted Average (MFN Weighted Average from United States by country.xlsx)
  - Simple Average (MFN Simple Average from United States by country.xlsx)

*Weighted = trade-volume adjusted; Simple = unweighted mean.*

Load four MFN datasets for initial EDA; decide later which measure to use based on analysis.

In [49]:
# get the columns to keep using the function
# this is passed as arugument to the clean_secondary_data function while cleaning all four mfn files
required_columns = get_cols_to_keep(2008,2022)

In [50]:
# read excel file to get MFN simple avg from United States by country
on_us_simple_avg = pd.read_excel(
    io = "./../../data/raw/secondary/MFN Simple Average from United States by country.xlsx",
    sheet_name="Partner-Timeseries")
print("Raw data")
print(on_us_simple_avg.head(3))

# clean the data
cleaned_on_us_simple_avg = clean_secondary_data(
    df=on_us_simple_avg,
    country_col="Reporter Name",
    value_col="mfn_on_us_simple_avg",
    cols_to_keep=required_columns)

# print the cleaned data
print("Cleaned data")
print(cleaned_on_us_simple_avg.head(3))

Raw data
  Reporter Name   Partner Name Trade Flow   Product Group  \
0   Afghanistan  United States     Import    All Products   
1       Albania  United States     Import    All Products   
2       Algeria  United States     Import    All Products   

                Indicator  1988  1989  1990  1991  1992  ...      2013  \
0  MFN Simple Average (%)   NaN   NaN   NaN   NaN   NaN  ...       NaN   
1  MFN Simple Average (%)   NaN   NaN   NaN   NaN   NaN  ...  4.325249   
2  MFN Simple Average (%)   NaN   NaN   NaN   NaN   NaN  ...       NaN   

        2014       2015       2016       2017      2018      2019      2020  \
0        NaN        NaN        NaN        NaN  9.994845       NaN       NaN   
1   3.196099   3.160638   3.610572   3.130952  4.109742  4.109742  4.055282   
2  16.599815  16.564086  16.321002  16.131292       NaN       NaN       NaN   

       2021  2022  
0       NaN   NaN  
1  3.883333   1.0  
2       NaN   NaN  

[3 rows x 40 columns]


European Union not found in regex
Occ.Pal.Terr not found in regex
Serbia, FR(Serbia/Montenegro) not found in regex


Cleaned data
  standardized_country  Year  mfn_on_us_simple_avg
0          Afghanistan  2008              7.680412
1              Albania  2008              5.974834
2              Algeria  2008             16.759215


In [51]:
# read excel file to get MFN weighted avg from United States by country
on_us_weighted_avg = pd.read_excel(
    io="./../../data/raw/secondary/MFN Weighted Average from United States by country.xlsx",
    sheet_name="Partner-Timeseries")
print("Raw data")
print(on_us_weighted_avg.head(3))

# clean the data
cleaned_on_us_weighted_avg = clean_secondary_data(
    df=on_us_weighted_avg,
    country_col="Reporter Name",
    value_col="mfn_on_us_weighted_avg",
    cols_to_keep=required_columns)

# print the cleaned data
print("Cleaned data")
print(cleaned_on_us_weighted_avg.head(3))

European Union not found in regex
Occ.Pal.Terr not found in regex
Serbia, FR(Serbia/Montenegro) not found in regex


Raw data
  Reporter Name   Partner Name Trade Flow   Product Group  \
0   Afghanistan  United States     Import    All Products   
1       Albania  United States     Import    All Products   
2       Algeria  United States     Import    All Products   

                  Indicator  1988  1989  1990  1991  1992  ...      2013  \
0  MFN Weighted Average (%)   NaN   NaN   NaN   NaN   NaN  ...       NaN   
1  MFN Weighted Average (%)   NaN   NaN   NaN   NaN   NaN  ...  3.403512   
2  MFN Weighted Average (%)   NaN   NaN   NaN   NaN   NaN  ...       NaN   

       2014      2015      2016      2017      2018      2019      2020  \
0       NaN       NaN       NaN       NaN  7.830689       NaN       NaN   
1  4.483685  4.483020  2.145006  2.024851  1.400979  1.400979  1.845249   
2  9.133752  8.737499  8.786167  8.862730       NaN       NaN       NaN   

       2021      2022  
0       NaN       NaN  
1  1.842675  0.112525  
2       NaN       NaN  

[3 rows x 40 columns]
Cleaned data
  standa

In [52]:
# read excel file to get United States MFN Simple Average by country
by_us_simple_avg = pd.read_excel(
    io="./../../data/raw/secondary/United States MFN Simple Average by country.xlsx",
    sheet_name="Partner-Timeseries")
print("Raw data")
print(by_us_simple_avg.head(3))

# clean the data
cleaned_by_us_simple_avg = clean_secondary_data(
    df=by_us_simple_avg,
    country_col="Partner Name",
    value_col="mfn_by_us_simple_avg",
    cols_to_keep=required_columns)

# print the cleaned data
print("Cleaned data")
print(cleaned_by_us_simple_avg.head(3))

Raw data
   Reporter Name            Partner Name Trade Flow   Product Group  \
0  United States                 Belgium     Import    All Products   
1  United States                   Benin     Import    All Products   
2  United States  British Virgin Islands     Import    All Products   

                Indicator      1991      1992      1993  1994      1995  ...  \
0  MFN Simple Average (%)       NaN       NaN       NaN   NaN       NaN  ...   
1  MFN Simple Average (%)  5.909259  7.970829  6.804545   NaN  5.261765  ...   
2  MFN Simple Average (%)  2.125133  2.767889  5.870973   NaN  4.847286  ...   

       2013      2014      2015      2016      2017      2018      2019  \
0  3.609070  3.697747  3.708574  3.645280  3.517781  3.753910  3.775372   
1  0.730266  1.445417  1.681967  1.352135  3.373258  5.023310  4.696279   
2  3.126760  2.246390  3.503256  3.052993  3.493301  4.201638  3.740384   

       2020      2021      2022  
0  3.656216  3.728462  3.720645  
1  3.973499  3.2

Serbia, FR(Serbia/Montenegro) not found in regex
Occ.Pal.Terr not found in regex


Cleaned data
     standardized_country  Year  mfn_by_us_simple_avg
0                 Belgium  2008              3.971776
1                   Benin  2008              2.076978
2  British Virgin Islands  2008              3.591914


In [53]:
# read excel file to get United States MFN Weighted Average by country
by_us_weight_avg = pd.read_excel(
    io="./../../data/raw/secondary/United States MFN Weighted Average by country.xlsx",
    sheet_name="Partner-Timeseries")
print("Raw data")
print(by_us_weight_avg.head(3))

# clean the data
cleaned_by_us_weight_avg = clean_secondary_data(
    df=by_us_weight_avg,
    country_col="Partner Name",
    value_col="mfn_by_us_weighted_avg",
    cols_to_keep=required_columns)

# print the cleaned data
print("Cleaned data")
print(cleaned_by_us_weight_avg.head(3))

Serbia, FR(Serbia/Montenegro) not found in regex
Occ.Pal.Terr not found in regex


Raw data
   Reporter Name Partner Name Trade Flow   Product Group  \
0  United States      Anguila     Import    All Products   
1  United States      Armenia     Import    All Products   
2  United States     Barbados     Import    All Products   

                  Indicator      1991      1992      1993  1994      1995  \
0  MFN Weighted Average (%)  1.904222  4.400452  0.526695   NaN  1.168822   
1  MFN Weighted Average (%)       NaN  1.011925  4.424968   NaN  8.613624   
2  MFN Weighted Average (%)  8.171513  7.663270  7.519867   NaN  6.204946   

   ...      2013      2014      2015      2016      2017      2018      2019  \
0  ...  0.498757  0.557963  0.512569  0.909911  0.629398  0.460614  1.009472   
1  ...  4.925550  4.972215  4.824073  4.978689  4.634624  5.724910  5.612600   
2  ...  0.632499  5.635480  6.395908  6.647886  0.592126  3.986902  3.814374   

        2020      2021      2022  
0   0.643754  1.486165  2.029526  
1   5.357893  5.698418  4.759993  
2  13.021130  0

##### Merge all MFN dataframes to get MFN tariffs
Both weighted and simple, on the US and by the US, across countries and years.


In [54]:
# Join all dataframes on standardized_country and Year columns
final_tariff_df = (
    cleaned_by_us_simple_avg
    .merge(cleaned_by_us_weight_avg, on=["standardized_country", "Year"], how="outer")
    .merge(cleaned_on_us_simple_avg, on=["standardized_country", "Year"], how="outer")
    .merge(cleaned_on_us_weighted_avg, on=["standardized_country", "Year"], how="outer")
)

final_tariff_df.head(5)

,standardized_country,Year,mfn_by_us_simple_avg,mfn_by_us_weighted_avg,mfn_on_us_simple_avg,mfn_on_us_weighted_avg
0,Afghanistan,2008,3.312432,1.639727,7.680412,5.483946
1,Afghanistan,2009,2.832641,1.424914,NaN,NaN
2,Afghanistan,2010,2.436634,1.149717,NaN,NaN
3,Afghanistan,2011,3.818663,1.362161,NaN,NaN
4,Afghanistan,2012,3.694554,1.522072,NaN,NaN


In [55]:
# write the final tariff dataframe to a csv file
final_tariff_df.to_csv("./../../data/processed/final_tariff_df.csv", index=False)

#### Merge Tariff and GDP Data

We merge the GDP datasets with MFN tariff datasets to create a final dataframe.
Each row represents a country–year pair, with GDP (current and 2015-adjusted) and
MFN tariff (simple and weighted, by US and on US) columns.


In [56]:
# merge GDP and Tarrif dfs on standardized_country and Year to get final dataframe
final_gdp_tariff_df = final_tariff_df.merge(
    final_gdp_df, on=["standardized_country", "Year"],
    how="outer")

# replace ".." with np.nan
final_gdp_tariff_df = final_gdp_tariff_df.replace("..", np.nan)


# write the final gdp and tarrif dataframe to a csv file
final_gdp_tariff_df.to_csv("./../../data/processed/final_gdp_tariff.csv", index=False)

# show first 5 rows of the final dataframe
final_gdp_tariff_df.head(5)

,standardized_country,Year,mfn_by_us_simple_avg,mfn_by_us_weighted_avg,mfn_on_us_simple_avg,mfn_on_us_weighted_avg,GDP,GDP_2015_adj
0,Afghanistan,2008,3.312432,1.639727,7.680412,5.483946,10109297047.5432,11060395115.6809
1,Afghanistan,2009,2.832641,1.424914,NaN,NaN,12416152732.0567,13426272073.2398
2,Afghanistan,2010,2.436634,1.149717,NaN,NaN,15856668555.8336,15354612541.5059
3,Afghanistan,2011,3.818663,1.362161,NaN,NaN,17805098206.3141,15420077665.5323
4,Afghanistan,2012,3.694554,1.522072,NaN,NaN,19907329777.5872,17386490239.3536


In [57]:
# # checking nulls in major patner countries

# list of major partner countries
major_countries = [
    "China", "Germany", "Japan", "United Kingdom", "India", "France", 
    "Italy", "Canada", "South Korea", "Russia", "Brazil", "Australia", 
    "Spain", "Mexico", "Indonesia", "Netherlands", "Saudi Arabia", 
    "Turkey", "Switzerland", "Argentina", "Sweden", "Poland", 
    "Belgium", "Thailand", "Austria", "Norway", "United Arab Emirates"
]

major_countries_standardized = standardized_country(major_countries)
df = final_gdp_tariff_df.replace("..", np.nan)
df.tail(10)
df = df[df["standardized_country"].isin(major_countries_standardized)]
df[df.isnull().any(axis=1)]

,standardized_country,Year,mfn_by_us_simple_avg,mfn_by_us_weighted_avg,mfn_on_us_simple_avg,mfn_on_us_weighted_avg,GDP,GDP_2015_adj
634,China,2012,3.691516,2.946727,NaN,NaN,8673664713189.24,9104482675356.36
635,China,2013,3.684973,2.946263,NaN,NaN,9743124247267.24,9812695262900.89
1446,India,2014,3.870291,3.379643,NaN,NaN,2039126479154.52,1947834564908.95
1461,Indonesia,2014,4.384821,5.601702,NaN,NaN,890814755533.537,820828013230.588
1462,Indonesia,2015,4.262896,5.896428,NaN,NaN,860854232686.214,860854232686.214
1991,Mexico,2019,3.726360,3.788846,NaN,NaN,1304106203902.45,1277682077998.64
1992,Mexico,2020,3.681595,3.733430,NaN,NaN,1121064767401.51,1170944075562.06
1993,Mexico,2021,3.678514,3.737552,NaN,NaN,1316569466735.28,1241768434263.7
2594,Russia,2022,3.547705,1.014466,NaN,NaN,2291612121334.64,1484605556957.56
2672,Saudi Arabia,2010,4.425942,3.537168,NaN,NaN,528207466666.667,523453786400.721
